In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, confusion_matrix
from keras.utils import plot_model

In [ ]:
# Generate some sample data for the autoencoder
x_train = np.random.rand(1000, 12)
x_test = np.random.rand(100, 12)

In [ ]:
# Define the input layer for the autoencoder
input_layer = keras.Input(shape=(12,))

# Define the encoder layers
encoded = keras.layers.Dense(8, activation='relu')(input_layer)
encoded = keras.layers.Dense(4, activation='relu')(encoded)

# Define the decoder layers
decoded = keras.layers.Dense(8, activation='relu')(encoded)
decoded = keras.layers.Dense(12, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = keras.Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Plot the model structure
plot_model(autoencoder, to_file='DAGMM.png', show_shapes=True, show_layer_names=True)

# Train the model
history = autoencoder.fit(x_train, x_train, epochs=100, batch_size=32, validation_data=(x_test, x_test))

# Plot the model loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Use the trained autoencoder to encode the training data
encoded_data = autoencoder.encoder(x_train).numpy()

# Fit a Gaussian Mixture Model with 2 components to the encoded training data
gmm = GaussianMixture(n_components=2)
gmm.fit(encoded_data)

In [ ]:
# Use the GMM to predict the probability densities for the encoded test data
encoded_test_data = autoencoder.encoder(x_test).numpy()
scores = gmm.score_samples(encoded_test_data)

# Identify anomalies based on the GMM probability densities
threshold = np.percentile(scores, 5)
anomalies = x_test[scores < threshold]

In [ ]:
# Calculate precision, recall, and F1 score
y_true = np.zeros_like(scores)
y_true[scores < threshold] = 1
y_pred = np.zeros_like(scores)
y_pred[scores < threshold] = 1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")

# Generate precision-recall curve with F1 score
precision, recall, thresholds = precision_recall_curve(y_true, scores)
f1_scores = 2 * precision * recall / (precision + recall)
plt.plot(recall, precision, label='Precision-Recall curve')
plt.plot(recall, f1_scores, label='F1 score')
plt.xlabel('Recall')
plt.ylabel('Precision/F1 score')
plt.legend()
plt.show()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Generate score with threshold graph
plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel('Threshold')
plt.legend()
plt.show()